In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as ply
%matplotlib inline

In [2]:
from DistributedOI.GraphTopology import Graph
import DistributedOI.data_prep as dp

In [4]:
# Define the type of the Graph
graphType='erdos_renyi'
p=0.03 # Network connectivity 0<p<=1
#graphType='ring'
#graphType='star'

In [5]:
number_of_nodes=20 # Number of node in a network
r=55   # Target dimension of the low rank subspace

# SVD_ImageNet32_data(200,32*32*3)
datasets="imagenet32"
#dataDimension=32*32*3
dataDimension=32*32

# Design Data 
eigengap=0.7
datasets="synthetic_n{}r{}g{}".format(number_of_nodes,r,int(gap*10))
data,TotalSamples,dataDimension=dp.create_centralized_data(dataDimension=20,TotalSamples=500*number_of_nodes,top_rank=r,eigen_gap=gap)
print('data:',data.shape)

# MNIST r = 59
datasets="mnist{}".format(number_of_nodes)
data,TotalSamples,dataDimension = dp.load_MNIST_data()
print('Mnist data:',data.shape)

# CIFAR10 r = 42
datasets="cifar{}".format(number_of_nodes)
data,TotalSamples,dataDimension = dp.load_CIFAR10_data()
print('cifar10 data:',data.shape)

# lfw data 55
datasets="lfw{}".format(number_of_nodes)
data,TotalSamples,dataDimension = dp.load_lfw_people()
print('lfw people data:',data.shape)

In [ ]:
# Distributed data to nodes
dp.distributed_data(datasets,data,number_of_nodes,TotalSamples)

In [ ]:
# Load local test
local_data,NumSamples,dataDimension=dp.load_local_data(datasets,0)
print(dataDimension,NumSamples)

In [ ]:
# Find top r with svd
covariance_matrix = (1/(TotalSamples))*np.dot(data,data.transpose())
top_r_eigenvec=dp.SVD(datasets=datasets,covariance_matrix=covariance_matrix, dimension=dataDimension, r=r)
print('Sample covariance matrix has shape :',covariance_matrix.shape)
print('Top r low rank subspace has shape :',top_r_eigenvec.shape)

In [ ]:
g=Graph(graphType=graphType,number_of_nodes=number_of_nodes,probability=p)
g.create_graph()
W=g.weight_matrix() # Grab the weight matrix
with open("Data/Test/{}/weight_{}.pickle".format(datasets,graphType), 'wb') as handle:
    pickle.dump(W, handle)
                
#distributed_covariance= dp.distributed_covariance(data,number_of_nodes,TotalSamples)

In [ ]:
# Initialize the initial value of the estimation
X_init = np.random.rand(dataDimension,r)
X_init, R = np.linalg.qr(X_init)

with open("Data/Test/{}/X_init.pickle".format(datasets), 'wb') as handle:
    pickle.dump(X_init, handle)